In [9]:
# from pprint import pprint
# import instagram_scraper

# args = {"login_user": "tebogomafikeng@gmail.com", "login_pass": "maftowncode17"}

# insta_scraper = instagram_scraper.InstagramScraper(**args)
# insta_scraper.authenticate_with_login()
# shared_data = insta_scraper.get_shared_data_userinfo(username='coursera')

# arr = []

# for item in insta_scraper.query_media_gen(shared_data):
#     arr.append(item)

# pprint(arr)

In [ ]:
# ! pip3 install instagram-scraper

In [56]:
import pandas as pd
import glob
import json
import re
from datetime import datetime
import emoji
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from pytz import timezone
# from google.cloud import language as lg
# from google.cloud.language import enums
# from google.cloud.language import types
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

In [2]:
 #See if existing user credential file exists or create one 
# try:
#     f= open("Instagram.txt","r")
#     contents = f.read()
#     username = contents.replace("=",",").split(",")[1]
#     password = contents.replace("=",",").split(",")[3]
# except:
#     f= open("Instagram.txt","w+")
#     username = input('Enter your Instagram username: ')
#     password = input('Enter your Instagram password: ')

In [3]:
# ! instagram-scraper -f ig_users.txt -u username -p password  --media-metadata --profile-metadata -m 5000 -d output1

In [2]:
json_files = glob.glob("output/*.json")
merged =[]

for file in json_files:
    with open(file, 'r') as f:
        json_data = json.load(f)
        if json_data:
            merged.append(json_data)

In [17]:
data = pd.DataFrame(columns=['Id','post','post_type','tags','likes','comments','views','date','sentiment','followers'])

In [18]:
stop_words = set(stopwords.words('english')) 
  
def filter_stop(txt):
    txt_tokens=word_tokenize(txt)
    txt_tokens=[word for word in txt_tokens if word not in stop_words]
    return ' '.join(txt_tokens)

def strip_emoji(text):
    new_text = re.sub(emoji.get_emoji_regexp(), r"", text)
    return new_text

In [66]:
# Loop through each JSON string in the list, 
# apply functions previously define and save all cleaned data in dataframe

follower_dict = {}
for file_d in merged:
    ig_id=''
    info_dict={}
    for key in file_d:
        if key=='GraphImages':
            for item in file_d[key]:
                try:
                    post_content=strip_emoji(re.sub('\n','',item['edge_media_to_caption']['edges'][0]['node']['text']))
                    info_dict['post']=filter_stop(post_content)
                    info_dict['Id']=item['username']
                    ig_id=item['username']
                    info_dict['likes']=item['edge_media_preview_like']['count']
                    
                    now_utc = datetime.now(timezone('UTC'))
                    
                    info_dict['date']=item['taken_at_timestamp']
                    info_dict['sentiment']=''
                    info_dict['comments']=item['edge_media_to_comment']['count']
                    info_dict['views'] = item['video_view_count']
                    info_dict['tags'] = item['tags']
                    info_dict['post_type'] = item['is_video']
                    
                    if info_dict['post_type'] == True:
                        info_dict['post_type'] = 'Video'
                    else:
                        info_dict['post_type'] = 'Image'
                    data = data.append(info_dict,ignore_index=True)
                    
                except:
                    continue
            
        if key=='GraphProfileInfo':
            follower_dict[ig_id]=file_d[key]['info']['followers_count']

data['followers']=data['Id'].map(follower_dict)

<ipython-input-18-f30c96f687aa>:9: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  new_text = re.sub(emoji.get_emoji_regexp(), r"", text)


In [67]:
data.tail()

,Id,post,post_type,tags,likes,comments,views,date,sentiment,followers
14848,explore_datascience,DAX makes Power BI powerful ! It used Power BI...,Video,"[dataanalysis, datascience, powerbi, elearning...",70,2,670,1643876360,,13584
14849,explore_datascience,Our students learning use Power BI create data...,Video,"[dataanalysis, datascience, graphs, datavisual...",28,2,462,1643699935,,13584
14850,explore_datascience,This week students focusing data visualisation...,Video,"[visualstorytelling, datascience, elearning, c...",31,0,357,1643115122,,13584
14851,explore_datascience,You option choose Data Science Data Engineerin...,Video,"[datascience, elearning, bigdata, dataengineer...",29,3,483,1641542114,,13584
14852,explore_datascience,There week remaining enrol Data Science course...,Video,"[sql, africa, enrolnow, datascience, python, m...",23,1,444,1641479664,,13584


In [65]:
format = "%Y-%m-%d %H:%M:%S %Z%z"
now_utc = datetime.now(timezone('UTC'))
print(now_utc.strftime(format))

# Convert to Asia/Kolkata time zone
now_asia = now_utc.astimezone(timezone('ZAF/Pretoria'))
print(now_asia.strftime(format))

timezone(unicode('EAT/Eastern African Time'))

2022-04-15 12:09:17 UTC+0000


UnknownTimeZoneError: 'ZAF/Pretoria'

In [72]:
#Exporting as csv file to program folder
data.to_csv("{}_posts.csv".format('instagram'), encoding='utf-8', index=False)

In [73]:
df = pd.read_csv('instagram_posts.csv')

In [74]:
df

,Id,post,post_type,tags,likes,comments,views,date,sentiment,followers
0,hyperiondev,Do know Career Services team offers ? The care...,NaN,NaN,23,1,NaN,2022-03-23 17:24:59,NaN,5433
1,hyperiondev,It 's month love ! And want share students Hyp...,NaN,NaN,28,1,NaN,2022-02-19 11:26:57,NaN,5433
2,hyperiondev,It 's Valentine 's Day ! And want share love c...,NaN,NaN,16,2,NaN,2022-02-14 17:32:36,NaN,5433
3,hyperiondev,Moipati way become multi-skilled coder web dev...,NaN,NaN,18,1,NaN,2022-01-25 16:48:04,NaN,5433
4,hyperiondev,Start new year right way - HyperionDev Immersi...,NaN,NaN,19,1,NaN,2022-01-20 11:59:04,NaN,5433
...,...,...,...,...,...,...,...,...,...,...
14848,explore_datascience,DAX makes Power BI powerful ! It used Power BI...,Video,"['dataanalysis', 'datascience', 'powerbi', 'el...",70,2,670,1643876360,NaN,13584
14849,explore_datascience,Our students learning use Power BI create data...,Video,"['dataanalysis', 'datascience', 'graphs', 'dat...",28,2,462,1643699935,NaN,13584
14850,explore_datascience,This week students focusing data visualisation...,Video,"['visualstorytelling', 'datascience', 'elearni...",31,0,357,1643115122,NaN,13584
14851,explore_datascience,You option choose Data Science Data Engineerin...,Video,"['datascience', 'elearning', 'bigdata', 'datae...",29,3,483,1641542114,NaN,13584
